In [1]:
import json
import pandas as pd
from pandas import json_normalize

# Load the JSON data
with open('masterjson.json', 'r') as file:
    data = json.load(file)


In [2]:
for item in data['results']:
    # Merge the attributes from 'attrs' into the main item
    item.update(item['attrs'])
    # Delete the now unnecessary keys
    del item['attrs']
    del item['id']
    del item['weight']
    del item['geom_quadindex']
    del item['geom_st_box2d']
    del item['origin']
    del item['rank']
    del item['x']
    del item['y']
    del item['zoomlevel']
    del item['num']
    del item['detail']

In [3]:
nodes_data = data['results']

# Convert to DataFrame
df = pd.DataFrame(nodes_data)

print(df.columns)


Index(['label', 'lat', 'lon', 'objectclass', 'Kategorie', 'schoolshort',
       'name', 'quarter', 'city', 'address', 'zipcode'],
      dtype='object')


In [4]:
data_dict = df.to_dict(orient='records')



In [5]:
### nur als proof of concept kann übersprungen werdem
import pandas as pd

def finde_gemeinde(plz):
    dateiname = 'PLZO_CSV_LV95.csv'
    df = pd.read_csv(dateiname, encoding='utf-8', delimiter=';')

    # Ensure PLZ column is of type int
    df['PLZ'] = df['PLZ'].astype(str).str.strip().astype(int)
    
    matching_rows = df[df['PLZ'] == plz]['Gemeindename']

    if matching_rows.empty:
        return "Unbekannt"

    return matching_rows.values[0]

# Test der Funktion
postleitzahl = 1702
print(f"Die Gemeinde für die Postleitzahl {postleitzahl} ist {finde_gemeinde(postleitzahl)}.")



Die Gemeinde für die Postleitzahl 1702 ist Unbekannt.


In [6]:
import pandas as pd
import json

def finde_gemeinde(plz):
    """
    Diese Funktion sucht nach einer Gemeinde basierend auf einer gegebenen Postleitzahl (PLZ).

    :param plz: Die Postleitzahl, für die die Gemeinde gesucht werden soll.
    :return: Den Namen der Gemeinde, die der gegebenen PLZ entspricht, oder None, wenn keine gefunden wird.
    """

    # Dateiname der CSV-Datei, die die PLZ und Gemeindenamen enthält.
    dateiname = 'PLZO_CSV_LV95.csv'

    # Die CSV-Datei wird in einen Pandas DataFrame geladen.
    # Das Einlesen der Datei bei jedem Aufruf ist nicht besonders effizient, aber die Datenmenge ist wahrscheinlich zu gering, um einen spürbaren Unterschied zu machen.
    df = pd.read_csv(dateiname, encoding='utf-8', delimiter=';')

    # Suche nach der Gemeinde, die der gegebenen PLZ entspricht.
    gemeinde = df[df['PLZ'] == int(plz)]['Gemeindename'].values

    # Wenn eine Gemeinde gefunden wird, wird ihr Name zurückgegeben.
    if len(gemeinde) > 0:
        return gemeinde[0]
    # Wenn keine Gemeinde gefunden wird, wird None zurückgegeben.
    else:
        return None

# Das ursprüngliche Dictionary mit den Hochschuldaten.
colleges = data_dict

# Eine leere Liste, um die aktualisierten Hochschuldaten zu speichern.
updated_colleges = []

# Durch jeden Eintrag im colleges-Dictionary iterieren.
for college in colleges:
    # Überprüfen, ob der Stadtnamen "Unknown City" hat.
    if college['city'] == "Unknown City":
        plz = college['zipcode']
        # Die Gemeinde basierend auf der PLZ suchen.
        city_name = finde_gemeinde(plz)
        # Wenn eine gültige Gemeinde gefunden wird, wird der Stadtnamen im college-Eintrag aktualisiert und der aktualisierte Eintrag wird der updated_colleges-Liste hinzugefügt.
        if city_name:
            college['city'] = city_name
            updated_colleges.append(college)
    # Wenn der Stadtnamen bereits bekannt ist, wird der Eintrag unverändert der updated_colleges-Liste hinzugefügt.
    else:
        updated_colleges.append(college)

# Das aktualisierte Dictionary wird in data_dict gespeichert.
data_dict = updated_colleges



In [7]:

colleges_clean = data_dict
# Zählen, wie oft "Unknown City" vorkommt
unknown_city_count = sum(1 for college in colleges_clean if college['city'] == "Unknown City")

print(f"'Unknown City' erscheint {unknown_city_count} Mal in der colleges_clean.json Datei.")


'Unknown City' erscheint 0 Mal in der colleges_clean.json Datei.


In [8]:
import json

def simplify_string(s):
    """
    Diese Funktion nimmt einen String `s` entgegen und entfernt bestimmte HTML-Tags, um den String zu vereinfachen.

    :param s: Der Eingabe-String, der bearbeitet werden soll.
    :return: Ein vereinfachter String ohne bestimmte HTML-Tags.
    """

    # Den Startindex für den zu entfernenden Bereich finden. Dieser liegt direkt nach dem ersten "<i>"-Tag.
    start_index = s.find('<i>') + 3

    # Den Endindex für den zu entfernenden Bereich finden. Dieser liegt direkt vor dem ersten "</i>"-Tag.
    end_index = s.find('</i>')

    # Den Bereich zwischen den "<i>" und "</i>"-Tags aus dem String entfernen.
    result = s.replace(s[start_index:end_index], '')

    # Alle "<b>" und "</b>"-Tags aus dem resultierenden String entfernen.
    result = result.replace('<b>', '').replace('</b>', '').replace('<i>', '').replace('</i>','')

    # Den bereinigten String zurückgeben, nachdem überflüssige Leerzeichen entfernt wurden.
    return result.strip()

def clean_labels_in_json(data_dict, file_path=None):
    """
    Diese Funktion nimmt ein Dictionary `data_dict` oder einen Dateipfad `file_path` zu einer JSON-Datei entgegen und
    bereinigt die 'label'-Einträge in jedem Element des JSON.

    :param data_dict: Das Eingabe-Dictionary, das bearbeitet werden soll.
    :param file_path: Der Pfad zur JSON-Datei, die bearbeitet werden soll.
    :return: Ein bereinigtes Dictionary.
    """

    # Wenn ein Dateipfad angegeben wurde, wird die JSON-Datei geöffnet und als Dictionary geladen.
    if file_path is not None:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
    else:
        # Wenn kein Dateipfad angegeben wurde, wird das übergebene Dictionary verwendet.
        data = data_dict

    # Über jedes Element im Dictionary iterieren.
    for item in data:
        # Den 'label'-Eintrag des aktuellen Elements mit der `simplify_string`-Funktion bereinigen.
        item['label'] = simplify_string(item['label'])

    # Das bereinigte Dictionary zurückgeben.
    return data

# Die `clean_labels_in_json`-Funktion wird auf das `data_dict` angewendet, um die 'label'-Einträge zu bereinigen.
data_dict = clean_labels_in_json(data_dict)


In [9]:
# Specify the path where you want to save the JSON file
output_path = 'colleges_clean.json'

# Save the dictionary as a JSON file
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(data_dict, file, indent=4, ensure_ascii=False)